In [1]:
from io import BytesIO
import numpy as np
import h5py
import png
import google.cloud.storage
from google.cloud import storage
import io
import matplotlib.pyplot as plt
import cv2
import os
import time
from IPython.display import clear_output
from PIL import Image
import pandas as pd
import octane_research_api.octane.api.core as octane

In [5]:
BUCKET_NAME = "research-datasets-3e92f8dd-ds-101"
INPUT_FOLDER = "source" 
OUTPUT_FOLDER = "outputs"

## Dataset interrogation and download of central OCT slices

In [3]:
storage_client = storage.Client()

def get_bytes(bucket_name, file):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file)
    return io.BytesIO(blob.download_as_string())

In [4]:
def visualise_OCT(path, slice):
    client = storage.Client()
    bucket = client.get_bucket(BUCKET_NAME) 
    blob = bucket.get_blob(path)  
    bytesIO = io.BytesIO(blob.download_as_string())
    h5 = h5py.File(bytesIO, 'r')
    h5_oct = h5['oct']
    plt.figure()
    plt.imshow(h5_oct[slice])
    return plt.show()

In [6]:
model_outputs = pd.read_csv(get_bytes(BUCKET_NAME, os.path.join(OUTPUT_FOLDER, "model_outputs_summary.csv")))

In [7]:
model_outputs.head()

,scan_name,model_output_path,model_version,urgency,urgency_probs_urgent,urgency_probs_semi-urgent,urgency_probs_routine,urgency_probs_observation,condition_probs_normal,condition_probs_macular_oedema,...,segmentation_4_volume_rpe,segmentation_4_volume_drusenoid_ped,segmentation_4_volume_serous_ped,segmentation_4_volume_fibrovascular_ped,segmentation_4_volume_choroid_and_outer_layers,segmentation_4_volume_mirror_artefact,segmentation_4_volume_clipping_artefact,segmentation_4_volume_blink_artefact,segmentation_4_thickness_central,status
0,0001/0001-oct-L-0-0,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,routine,0.092,0.061,0.702,0.145,0.454,0.060,...,0.870,0.000,0.0,0.02,47.589,0.0,9.350,3.075,273.824,success
1,0001/0001-oct-R-0-0,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,observation,0.030,0.023,0.128,0.819,0.963,0.031,...,0.987,0.000,0.0,0.00,39.314,0.0,14.621,0.000,268.257,success
2,0001/0001-oct-R-0-1,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,observation,0.045,0.078,0.299,0.578,0.879,0.056,...,1.020,0.000,0.0,0.00,49.268,0.0,4.204,0.000,253.967,success
3,0001/0001-oct-R-0-2,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,observation,0.031,0.029,0.164,0.776,0.952,0.031,...,1.027,0.000,0.0,0.00,50.468,0.0,3.617,0.000,273.681,success
4,0002/0002-oct-L-0-0,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,observation,0.120,0.072,0.291,0.517,0.872,0.056,...,0.805,0.006,0.0,0.00,51.022,0.0,4.575,0.000,260.198,success


In [10]:
drusen_ds_101 = model_outputs.loc[model_outputs['condition_probs_drusen'] >= 0.95]

In [9]:
drusen_ds_101 = drusen_ds_101.reset_index()

In [11]:
drusen_ds_101.head()

,scan_name,model_output_path,model_version,urgency,urgency_probs_urgent,urgency_probs_semi-urgent,urgency_probs_routine,urgency_probs_observation,condition_probs_normal,condition_probs_macular_oedema,...,segmentation_4_volume_rpe,segmentation_4_volume_drusenoid_ped,segmentation_4_volume_serous_ped,segmentation_4_volume_fibrovascular_ped,segmentation_4_volume_choroid_and_outer_layers,segmentation_4_volume_mirror_artefact,segmentation_4_volume_clipping_artefact,segmentation_4_volume_blink_artefact,segmentation_4_thickness_central,status
7,0003/0003-oct-R-0-0,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,routine,0.236,0.103,0.491,0.170,0.580,0.092,...,0.896,0.002,0.0,0.000,13.870,0.000,17.305,0.000,212.022,success
24,0009/0009-oct-L-0-0,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,urgent,0.478,0.090,0.421,0.011,0.082,0.101,...,0.192,0.000,0.0,0.000,35.818,0.000,4.999,0.000,305.156,success
25,0009/0009-oct-R-0-0,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,urgent,0.476,0.117,0.399,0.007,0.083,0.134,...,0.164,0.002,0.0,0.018,25.685,0.005,7.604,3.741,204.822,success
26,0009/0009-oct-R-0-1,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,routine,0.261,0.060,0.671,0.008,0.087,0.075,...,0.207,0.000,0.0,0.052,30.680,0.000,11.212,0.000,273.013,success
40,0015/0015-oct-L-0-0,/bigstore/research-datasets-3e92f8dd-ds-101/ou...,0.9.4-topcon-nm-1gpu,routine,0.114,0.036,0.839,0.011,0.093,0.061,...,0.923,0.256,0.0,0.000,35.090,0.000,8.402,0.000,246.015,success


In [13]:
print(drusen_ds_101.iloc[100].condition_probs_drusen)

0.9540000000000001


In [14]:
drusen_ds_101.shape

(749, 127)

### DS-101 image harvesting

In [15]:
saved_file_name = 0

#ds_40_oct_download:
for i, row in drusen_ds_101.iterrows():
    scan_name = drusen_ds_101['scan_name'][i]
    scan = octane.Scan(h5py.File(get_bytes(BUCKET_NAME, os.path.join(INPUT_FOLDER, f"{scan_name}.h5")), "r")['oct'][()])
    oct_slice = scan.voxels[64, :, :]
    png.from_array(oct_slice, 'L').save("DS_101_drusen/" + "Drusen_image_" + str(saved_file_name) + ".png")
    saved_file_name += 1

### DS-25 image harvesting

In [73]:
cyclegan_final_name = 0
IMG_SIZE = 512

path = os.path.join('Official_DS_40_images')

In [81]:
#ds_25_oct_donwload:
def download_oct_images_to_file(bucket_name, prefix, delimiter=None):

    storage_client = storage.Client()

    blobs = storage_client.list_blobs(
        bucket_name, prefix=prefix, delimiter=delimiter
    )
    
    print("Blobs:")
    
    file_name = 2
    count = 0
    
    for blob in blobs:
        if count <2:
            try:
                bytesIO = io.BytesIO(blob.download_as_string())
                h5 = h5py.File(bytesIO, 'r')
                h5_oct = h5['oct']
                png.from_array(h5_oct[64], 'L').save("Official_DS_40_images/" + "image_" + str(file_name) + ".png")
                count += 1
                file_name += 1
            except:
                pass

In [82]:
download_oct_images_to_file(BUCKET_NAME, 'files/0002/0002-oct-R-0-0.h5')

Blobs:


In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)